<a href="https://colab.research.google.com/github/realmistic/PythonInvest-basic-fin-analysis/blob/master/colab_notebooks/Part17_SNP500_Fundamental_Analysis_using_AlphaVantage.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Stock Screening Using Paid Data

## What?
This notebook demonstrates a step-by-step process for screening US stocks using advanced financial metrics from paid data sources. The goal is to systematically identify companies with strong profitability, growth, and attractive valuations.

## Why?
Paid data provides more accurate, timely, and comprehensive financial information than free sources. This enables investors to apply precise screening thresholds, benchmark companies against industry peers, and construct a robust, repeatable process for selecting high-quality stocks.

## How?
The screening workflow consists of four main steps:
1. **Data Acquisition:** Gather detailed financial and fundamental data for a broad universe of US stocks using a paid data provider.
2. **Data Preparation:** Clean, preprocess, and organize the data, including calculating or merging key financial metrics and industry benchmarks.
3. **Screening Logic:** Sequentially apply quantitative filters for profitability, growth, and value to systematically narrow down the stock universe.
4. **Results & Analysis:** Review and visualize the final shortlist of stocks that meet all criteria for further research or investment consideration.

## LLM Reasoning
Large Language Models (LLMs) can be integrated into this process to enhance transparency and interpretability. By generating step-by-step explanations or critiques for each screening stage, LLMs help clarify the rationale behind each filter and support more informed, data-driven investment decisions.

---

For methodology details, see the full article:  
https://pythoninvest.com/long-read/stock-screening-using-paid-data



In [ ]:
import os
import requests
import csv
import time
from tqdm import tqdm

# 1) Get the list of tickers from Wiki (S&P500)

In [ ]:
# prompt: i need to import this table to pandas https://en.wikipedia.org/wiki/List_of_S%26P_500_companies#S&P_500_component_stocks

import pandas as pd

# URL of the Wikipedia page
url = 'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies#S&P_500_component_stocks'

# Read the table from the Wikipedia page into a pandas DataFrame
tables = pd.read_html(url)

# Assuming the table we want is the first one on the page
tickers_df = tables[0]

# Display the DataFrame
tickers_df

,Symbol,Security,GICS Sector,GICS Sub-Industry,Headquarters Location,Date added,CIK,Founded
0,MMM,3M,Industrials,Industrial Conglomerates,"Saint Paul, Minnesota",1957-03-04,66740,1902
1,AOS,A. O. Smith,Industrials,Building Products,"Milwaukee, Wisconsin",2017-07-26,91142,1916
2,ABT,Abbott Laboratories,Health Care,Health Care Equipment,"North Chicago, Illinois",1957-03-04,1800,1888
3,ABBV,AbbVie,Health Care,Biotechnology,"North Chicago, Illinois",2012-12-31,1551152,2013 (1888)
4,ACN,Accenture,Information Technology,IT Consulting & Other Services,"Dublin, Ireland",2011-07-06,1467373,1989
...,...,...,...,...,...,...,...,...
498,XYL,Xylem Inc.,Industrials,Industrial Machinery & Supplies & Components,"White Plains, New York",2011-11-01,1524472,2011
499,YUM,Yum! Brands,Consumer Discretionary,Restaurants,"Louisville, Kentucky",1997-10-06,1041061,1997
500,ZBRA,Zebra Technologies,Information Technology,Electronic Equipment & Instruments,"Lincolnshire, Illinois",2019-12-23,877212,1969
501,ZBH,Zimmer Biomet,Health Care,Health Care Equipment,"Warsaw, Indiana",2001-08-07,1136869,1927


In [ ]:
# 503 entries
tickers_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 503 entries, 0 to 502
Data columns (total 8 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   Symbol                 503 non-null    object
 1   Security               503 non-null    object
 2   GICS Sector            503 non-null    object
 3   GICS Sub-Industry      503 non-null    object
 4   Headquarters Location  503 non-null    object
 5   Date added             503 non-null    object
 6   CIK                    503 non-null    int64 
 7   Founded                503 non-null    object
dtypes: int64(1), object(7)
memory usage: 31.6+ KB


In [ ]:
# What are the Sectors?
tickers_df['GICS Sector'].value_counts()

,count
GICS Sector,
Industrials,78
Financials,73
Information Technology,69
Health Care,60
Consumer Discretionary,51
Consumer Staples,38
Utilities,31
Real Estate,31
Materials,26


In [ ]:
# Sectors and subsectors?
tickers_df[['GICS Sector','GICS Sub-Industry']].value_counts()

GICS Sector             GICS Sub-Industry                           
Health Care             Health Care Equipment                           17
Utilities               Electric Utilities                              15
Industrials             Industrial Machinery & Supplies & Components    14
Information Technology  Semiconductors                                  14
                        Application Software                            13
                                                                        ..
Real Estate             Industrial REITs                                 1
                        Timber REITs                                     1
                        Single-Family Residential REITs                  1
Utilities               Gas Utilities                                    1
                        Water Utilities                                  1
Name: count, Length: 129, dtype: int64

In [ ]:
# Top Sub-sectors?
tickers_df[['GICS Sub-Industry']].value_counts().head(10)

,count
GICS Sub-Industry,
Health Care Equipment,17
Electric Utilities,15
Semiconductors,14
Industrial Machinery & Supplies & Components,14
Application Software,13
Multi-Utilities,12
Packaged Foods & Meats,12
Aerospace & Defense,12
Asset Management & Custody Banks,11


In [ ]:
# Get list of tickers to be used later for the Fundamental analysis
TICKERS = tickers_df.Symbol.unique().tolist()
# Exampel
TICKERS[0:5]

['MMM', 'AOS', 'ABT', 'ABBV', 'ACN']

# 2) Define Functions to Work with Alpha Vantage

In [ ]:
# Function to fetch data from Alpha Vantage API
# Endpoint: https://www.alphavantage.co/documentation/#company-overview

def fetch_stock_data(symbol, api_key):
    url = f"https://www.alphavantage.co/query?function=OVERVIEW&symbol={symbol}&apikey={api_key}"
    response = requests.get(url)

    if response.status_code == 200:
        data = response.json()
        if data:
            return data
        else:
            print(f"No data found for symbol: {symbol}")
            return None
    else:
        print(f"Error fetching data for {symbol}: {response.status_code}")
        return None

In [ ]:
# Download the stats via API
def get_stocks_dataframe(tickers, api_key):
  all_stock_data = []

  # Fetch stock data for each ticker and add it to the list
  for ticker in tqdm(tickers, desc="Fetching company data"):
    ticker_adj = ticker.replace('.', '-') # Example 'BRK.B' -> 'BRK-B'
    stock_data = fetch_stock_data(ticker_adj, api_key)
    if stock_data:
      all_stock_data.append(stock_data)
      time.sleep(1)  # Rate limiting

  return pd.DataFrame(all_stock_data)

# 3) Small Dataset with FREE API Key
* FREE API KEY (https://www.alphavantage.co/support/#api-key) with 25 calls per day (enough to cover 25 companies)


In [ ]:
from google.colab import userdata
FREE_API_KEY = userdata.get('ALPHA_VANTAGE_API_KEY')

In [ ]:
# Import file OR download it from the API
# import pandas as pd
# import os

# if os.path.exists('snp500_companies_fundamental_data_small.csv'):
#   df_short = pd.read_csv('snp500_companies_fundamental_data_small.csv')
# else:
#   df_short = get_stocks_dataframe(TICKERS[:25], FREE_API_KEY)
#   df_short.to_csv('snp500_companies_fundamental_data_small.csv', index=False)


In [ ]:
# # Top 25 tickers
# df_short = get_stocks_dataframe(TICKERS[0:25], FREE_API_KEY)

In [ ]:
# df_short.to_csv("snp500_companies_fundamental_data_small.csv")

In [ ]:
# df_short.head()

In [ ]:
# What are the available fields?
# df_short.keys()

# 4) Full Dataset (500 companies) with PAID KEY

In [ ]:
from google.colab import userdata
PAID_API_KEY = userdata.get('ALPHA_VANTAGE_PAID_API_KEY')

In [ ]:
# Import from File or generate a new one with PAID API (easy re-run for Colab)

import pandas as pd
import os

# Check if the file exists
if os.path.exists('snp500_companies_fundamental_data_full.csv'):
  df_full = pd.read_csv('snp500_companies_fundamental_data_full.csv')
  print("File found and loaded into df_full")
else:
  print("File not found. Generating a new file using Paid API.")
  df_full = get_stocks_dataframe(TICKERS, PAID_API_KEY)
  df_full.to_csv("snp500_companies_fundamental_data_full.csv")


File found and loaded into df_full


In [ ]:
# Full list of tickers
# df_full = get_stocks_dataframe(tickers = TICKERS, api_key=PAID_API_KEY)

In [ ]:
df_full.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 503 entries, 0 to 502
Data columns (total 53 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   Unnamed: 0                  503 non-null    int64  
 1   Symbol                      503 non-null    object 
 2   AssetType                   503 non-null    object 
 3   Name                        503 non-null    object 
 4   Description                 499 non-null    object 
 5   CIK                         503 non-null    int64  
 6   Exchange                    503 non-null    object 
 7   Currency                    503 non-null    object 
 8   Country                     503 non-null    object 
 9   Sector                      503 non-null    object 
 10  Industry                    503 non-null    object 
 11  Address                     503 non-null    object 
 12  OfficialSite                503 non-null    object 
 13  FiscalYearEnd               503 non

In [ ]:
df_full.head()

,Unnamed: 0,Symbol,AssetType,Name,Description,CIK,Exchange,Currency,Country,Sector,...,EVToRevenue,EVToEBITDA,Beta,52WeekHigh,52WeekLow,50DayMovingAverage,200DayMovingAverage,SharesOutstanding,DividendDate,ExDividendDate
0,0,MMM,Common Stock,3M Company,The 3M Company is an American multinational co...,66740,NYSE,USD,USA,LIFE SCIENCES,...,3.226,10.75,1.060,156.35,88.63,146.55,133.21,539318000,2025-03-12,2025-02-14
1,1,AOS,Common Stock,Smith AO Corporation,A. O. Smith Corporation is an American manufac...,91142,NYSE,USD,USA,MANUFACTURING,...,2.435,11.82,1.255,91.22,58.83,65.87,74.97,117659000,2025-05-15,2025-04-30
2,2,ABT,Common Stock,Abbott Laboratories,Abbott Laboratories is an American multination...,1800,NYSE,USD,USA,LIFE SCIENCES,...,5.460,21.22,0.801,140.58,98.26,130.79,117.78,1734320000,2025-05-15,2025-04-15
3,3,ABBV,Common Stock,AbbVie Inc,AbbVie is an American publicly traded biopharm...,1551152,NYSE,USD,USA,LIFE SCIENCES,...,6.720,25.38,0.538,216.66,148.19,200.18,187.97,1768980000,2025-05-15,2025-04-15
4,4,ACN,Common Stock,Accenture plc,Accenture plc is an Irish-domiciled multinatio...,1467373,NYSE,USD,USA,TRADE & SERVICES,...,2.692,15.28,1.315,396.38,273.65,338.60,344.11,626026000,2025-05-15,2025-04-10


In [ ]:
df_full.to_csv("snp500_companies_fundamental_data_full.csv")

In [ ]:
import pandas as pd

# Load DataFrame (Assuming data is in 'df')
df = df_full.copy(deep=True)

# Convert relevant columns to numeric, handling errors
for col in ['ReturnOnEquityTTM', 'QuarterlyEarningsGrowthYOY',
            'QuarterlyRevenueGrowthYOY', 'PEGRatio', 'EVToEBITDA', 'EVToRevenue',
            '50DayMovingAverage', '200DayMovingAverage', 'PriceToBookRatio',
            'Beta', 'AnalystRatingBuy', 'AnalystRatingSell',
            'AnalystRatingStrongSell', 'AnalystTargetPrice', 'MarketCapitalization','OperatingMarginTTM', 'ProfitMargin']:
    try:
        df[col] = pd.to_numeric(df[col], errors='coerce')
    except KeyError:
        print(f"Column '{col}' not found in the DataFrame.")


# Subset of used Columns
COLUMNS = ['Symbol', 'LatestQuarter', 'ReturnOnEquityTTM', 'QuarterlyEarningsGrowthYOY', 'QuarterlyRevenueGrowthYOY', 'MarketCapitalization', 'Beta', 'EVToEBITDA','PEGRatio',
           'TrailingPE','PERatio', 'Sector', 'ProfitMargin', 'EVToRevenue',
           'AnalystRatingBuy', 'AnalystRatingSell', 'AnalystTargetPrice', '50DayMovingAverage','200DayMovingAverage']


# 5) Analysis: few months to year investment selection strategy based on fundamental factors
**All available fields:**
'Symbol', 'AssetType', 'Name', 'Description', 'CIK', 'Exchange',
       'Currency', 'Country', 'Sector', 'Industry', 'Address', 'OfficialSite',
       'FiscalYearEnd', 'LatestQuarter', 'MarketCapitalization', 'EBITDA',
       'PERatio', 'PEGRatio', 'BookValue', 'DividendPerShare', 'DividendYield',
       'EPS', 'RevenuePerShareTTM', 'ProfitMargin', 'OperatingMarginTTM',
       'ReturnOnAssetsTTM', 'ReturnOnEquityTTM', 'RevenueTTM',
       'GrossProfitTTM', 'DilutedEPSTTM', 'QuarterlyEarningsGrowthYOY',
       'QuarterlyRevenueGrowthYOY', 'AnalystTargetPrice',
       'AnalystRatingStrongBuy', 'AnalystRatingBuy', 'AnalystRatingHold',
       'AnalystRatingSell', 'AnalystRatingStrongSell', 'TrailingPE',
       'ForwardPE', 'PriceToSalesRatioTTM', 'PriceToBookRatio', 'EVToRevenue',
       'EVToEBITDA', 'Beta', '52WeekHigh', '52WeekLow', '50DayMovingAverage',
       '200DayMovingAverage', 'SharesOutstanding', 'DividendDate',
       'ExDividendDate'

**Filter Companies Sequentially**
* 1) Fundamentals: Profitability & Growth (ROE, Earnings/Revenue YoY, Margin)
* 2) Valuation Metrics (PEG, EV/EBITDA)
* 3) Technical Momentum (50d MA> 200d MA, beta)
* 4) Analyst Sentiment (target price>currentPrice)
* Final Selection of Top 3 Stocks

## Step1 : Fundamentals: Profitability & Growth
* 'ReturnOnEquityTTM',
* 'QuarterlyEarningsGrowthYOY',
* 'QuarterlyRevenueGrowthYOY'
* 'OperatingMarginTTM' or 'Profit Margin'

In [ ]:
# Shape of the distributions in 1 table
df[['ReturnOnEquityTTM','QuarterlyEarningsGrowthYOY','QuarterlyRevenueGrowthYOY','OperatingMarginTTM','ProfitMargin']].describe().T

,count,mean,std,min,25%,50%,75%,max
ReturnOnEquityTTM,503.0,0.224710,0.431291,-5.610,0.0849,0.157,0.2880,2.761
QuarterlyEarningsGrowthYOY,503.0,0.677773,4.130577,-0.999,-0.1580,0.074,0.3695,79.880
QuarterlyRevenueGrowthYOY,503.0,0.067083,0.162287,-0.656,0.0020,0.051,0.1165,1.632
OperatingMarginTTM,503.0,0.201510,0.157911,-1.290,0.1140,0.187,0.2790,0.851
ProfitMargin,503.0,0.141309,0.144235,-1.100,0.0729,0.129,0.2020,0.809


In [ ]:
# Dynamic graphs with Plotly Express

import plotly.express as px
fig = px.histogram(df[df.ReturnOnEquityTTM.between(0,1)], x='ReturnOnEquityTTM', nbins=100)
fig.show()

In [ ]:
# When returns are negative?
df[df.ReturnOnEquityTTM<0][COLUMNS].sort_values(by='MarketCapitalization', ascending=False)


,Symbol,LatestQuarter,ReturnOnEquityTTM,QuarterlyEarningsGrowthYOY,QuarterlyRevenueGrowthYOY,MarketCapitalization,Beta,EVToEBITDA,PEGRatio,TrailingPE,PERatio,Sector,ProfitMargin,EVToRevenue,AnalystRatingBuy,AnalystRatingSell,AnalystTargetPrice,50DayMovingAverage,200DayMovingAverage
471,VRTX,2024-12-31,-0.0315,-0.056,0.157,128666460000,0.506,255.600,1.372,-,NaN,LIFE SCIENCES,-0.0486,11.280,16.0,1.0,498.01,486.14,470.75
70,BMY,2024-12-31,-0.3900,-0.959,0.075,101737996000,0.411,45.710,1.167,-,NaN,LIFE SCIENCES,-0.1850,2.998,4.0,1.0,59.86,57.90,53.53
129,CRWD,2025-01-31,-0.0059,5.330,0.252,97364509000,1.276,306.390,4.416,-,NaN,TECHNOLOGY,-0.0049,22.850,27.0,0.0,409.17,380.62,333.28
249,INTC,2024-12-31,-0.1790,-0.717,-0.074,86557712000,1.122,96.850,0.501,-,NaN,MANUFACTURING,-0.3530,2.194,2.0,1.0,22.68,22.28,23.10
452,TFC,2024-12-31,-0.0007,0.239,0.065,47633682000,0.891,NaN,1.944,-,NaN,FINANCE,0.4220,7.990,7.0,0.0,47.73,42.37,43.39
130,CCI,2024-12-31,-1.2490,0.142,-0.015,43765469000,0.952,20.710,2.784,-,NaN,REAL ESTATE & CONSTRUCTION,-0.5940,11.150,4.0,0.0,110.18,96.53,102.67
433,TTWO,2024-12-31,-0.5140,-0.497,-0.005,37876044000,1.010,49.450,1.447,-,NaN,TECHNOLOGY,-0.6710,7.480,17.0,1.0,216.93,207.48,176.12
119,STZ,2025-02-28,-0.0036,0.228,0.012,33233799000,0.789,57.680,1.254,-,NaN,MANUFACTURING,-0.0080,4.375,8.0,0.0,216.80,177.38,221.09
494,WTW,2024-12-31,-0.0100,1.059,0.042,32513423000,0.687,44.420,1.080,-,NaN,FINANCE,-0.0099,3.681,8.0,0.0,366.15,327.72,304.40
483,WBD,2024-12-31,-0.2820,2.267,-0.025,20251769000,1.432,4.689,2.786,-,NaN,TECHNOLOGY,-0.2880,1.385,10.0,1.0,13.27,10.19,9.13


In [ ]:
# Top Three big companies with negative ROE
df[df.ReturnOnEquityTTM<0][COLUMNS].head(3).T

,13,58,70
Symbol,ALB,BAX,BMY
LatestQuarter,2024-12-31,2024-12-31,2024-12-31
ReturnOnEquityTTM,-0.114,-0.0421,-0.39
QuarterlyEarningsGrowthYOY,-0.662,-0.945,-0.959
QuarterlyRevenueGrowthYOY,-0.477,0.01,0.075
MarketCapitalization,6354330000,14259287000,101737996000
Beta,1.595,0.647,0.411
EVToEBITDA,22.45,23.25,45.71
PEGRatio,0.99,1.808,1.167
TrailingPE,-,-,-


In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

def visualise_metrics_quantiles(df, metrics, is_percent=True):

# metrics = ['ReturnOnEquityTTM', 'QuarterlyEarningsGrowthYOY',
#            'QuarterlyRevenueGrowthYOY', 'OperatingMarginTTM', 'ProfitMargin']

  fig = make_subplots(rows=1, cols=len(metrics), shared_yaxes=True,
                      horizontal_spacing=0.03, subplot_titles=metrics)

  for i, metric in enumerate(metrics):
      metric_data = df[metric].dropna()

      # Trim to 2.5–97.5%
      lower = metric_data.quantile(0.025)
      upper = metric_data.quantile(0.975)
      trimmed = metric_data[(metric_data >= lower) & (metric_data <= upper)]

      # Compute quartiles (as %)
      q1 = trimmed.quantile(0.25)
      q2 = trimmed.quantile(0.50)
      q3 = trimmed.quantile(0.75)

      if is_percent==True: # (as %)
        q1 = q1 * 100
        q2 = q2 * 100
        q3 = q3 * 100

      # Convert to percent
      if is_percent==True:
        trimmed_percent = trimmed * 100
      else:
        trimmed_percent = trimmed

      fig.add_trace(go.Box(
          x=trimmed_percent,
          name=metric,
          boxpoints=False,
          boxmean=False,
          orientation='h',
          marker=dict(color='rgba(0, 128, 255, 0.5)'),
          line=dict(color='blue'),
          fillcolor='rgba(0, 128, 255, 0.2)'
      ), row=1, col=i+1)

      # if is_percent==True:
      #   ann = f"{label}: {val:.1f}%"
      # else:
      #   ann = f"{label}: {val:.2f}"

      # Annotations: Q1, Q2 (median), Q3
      if is_percent==True:
        for val, label, y_pos in zip([q1, q2, q3], ['Q1', 'Q2 (Median)', 'Q3'], [0.3, 0.5, 0.7]):
            fig.add_annotation(
                x=val, y=y_pos,
                text=f"{label}: {val:.1f}%",
                showarrow=False,
                font=dict(size=11, color='gray'),
                xanchor='center',
                row=1, col=i+1
            )
      else:
         for val, label, y_pos in zip([q1, q2, q3], ['Q1', 'Q2 (Median)', 'Q3'], [0.3, 0.5, 0.7]):
            fig.add_annotation(
                x=val, y=y_pos,
                text=f"{label}: {val:.1f}",
                showarrow=False,
                font=dict(size=11, color='gray'),
                xanchor='center',
                row=1, col=i+1
            )

  # Final layout
  fig.update_layout(
      height=500,
      width=300 * len(metrics),
      title_text="<b>Financial Metric Boxplots (Trimmed 2.5–97.5%, with Q1/Q2/Q3)</b>",
      title_x=0.5,
      template="plotly_white",
      showlegend=False,
  )

  # Format x-axis as percentage
  for i in range(1, len(metrics)+1):
      fig.update_xaxes(tickformat=".0f%", row=1, col=i)

  fig.show()
  return

In [ ]:
visualise_metrics_quantiles(df= df, metrics = ['ReturnOnEquityTTM', 'QuarterlyEarningsGrowthYOY', 'QuarterlyRevenueGrowthYOY', 'OperatingMarginTTM', 'ProfitMargin'])

In [ ]:
df[['ReturnOnEquityTTM','QuarterlyEarningsGrowthYOY','QuarterlyRevenueGrowthYOY','OperatingMarginTTM','ProfitMargin']].describe().T

,count,mean,std,min,25%,50%,75%,max
ReturnOnEquityTTM,503.0,0.224710,0.431291,-5.610,0.0849,0.157,0.2880,2.761
QuarterlyEarningsGrowthYOY,503.0,0.677773,4.130577,-0.999,-0.1580,0.074,0.3695,79.880
QuarterlyRevenueGrowthYOY,503.0,0.067083,0.162287,-0.656,0.0020,0.051,0.1165,1.632
OperatingMarginTTM,503.0,0.201510,0.157911,-1.290,0.1140,0.187,0.2790,0.851
ProfitMargin,503.0,0.141309,0.144235,-1.100,0.0729,0.129,0.2020,0.809


In [ ]:
# STEP 1: Profitability & Growth : select Q1(25%) or 0% values as the lower threshold

step1 = df[
    (df['ReturnOnEquityTTM'] > 0.085) &
    (df['QuarterlyEarningsGrowthYOY'] > 0.0) &
    (df['QuarterlyRevenueGrowthYOY'] > 0.0) &
    (df['OperatingMarginTTM'] > 0.114) &
    (df['ProfitMargin'] > 0.073)
]

In [ ]:
df.shape

(503, 53)

In [ ]:
step1.shape

(171, 53)

In [ ]:
print(f"After Profitability & Growth Filter: {step1.shape[0]} companies")
step1[['Symbol', 'LatestQuarter','MarketCapitalization', 'ReturnOnEquityTTM', 'QuarterlyEarningsGrowthYOY', 'QuarterlyRevenueGrowthYOY','OperatingMarginTTM']].\
    sort_values(by=['MarketCapitalization'], ascending=False).head().\
    style.format({'MarketCapitalization': '${:,.0f}'})


After Profitability & Growth Filter: 171 companies


,Symbol,LatestQuarter,MarketCapitalization,ReturnOnEquityTTM,QuarterlyEarningsGrowthYOY,QuarterlyRevenueGrowthYOY,OperatingMarginTTM
39,AAPL,2024-12-31,"$3,036,567,306,000",1.365000,0.101000,0.040000,0.345000
317,MSFT,2024-12-31,"$2,867,509,068,000",0.343000,0.102000,0.123000,0.455000
347,NVDA,2025-01-31,"$2,737,679,892,000",1.192000,0.836000,0.779000,0.611000
20,GOOG,2024-12-31,"$1,918,520,525,000",0.329000,0.309000,0.118000,0.340000
19,GOOGL,2024-12-31,"$1,918,377,001,000",0.329000,0.309000,0.118000,0.340000


## Step2 : Apply Valuation Metrics
* PEG Ratio < 1.5 (indicates growth at a reasonable price)
* EV/EBITDA < Industry Median (lower value means better relative valuation)

---

**📊 Understanding the PEG Ratio (< 1.5) in Stock Selection**

The **PEG (Price/Earnings-to-Growth) Ratio** is a **valuation metric** that adjusts the **P/E ratio** by a company's expected growth rate. It helps determine whether a stock is **fairly valued**, **overvalued**, or **undervalued** in relation to its earnings growth.


**📌 PEG Ratio Formula**
$$
PEG = \frac{P/E}{Earnings Growth (\%)}
$$
Where:  
- **P/E (Price-to-Earnings Ratio)** = Stock Price ÷ Earnings per Share (EPS)  
- **Earnings Growth (%)** = Expected or historical earnings growth rate  

 **📌 Why PEG < 1.5?**
- A **PEG ratio below 1** typically signals **undervaluation** (cheap relative to growth).  
- A **PEG ratio around 1 - 1.5** is considered **fair value**.  
- A **PEG ratio above 1.5 - 2+** may indicate **overvaluation** (too expensive for its growth rate).  

We set **PEG < 1.5** to filter for **reasonably priced growth stocks** that offer **upside potential without being overvalued**.

---

**📊 Understanding EV/EBITDA in Stock Selection**  

The **EV/EBITDA (Enterprise Value to Earnings Before Interest, Taxes, Depreciation, and Amortization)** ratio is a **valuation metric** that helps assess whether a stock is fairly priced compared to its earnings before non-cash expenses.  

It is commonly used for comparing companies across industries because it accounts for **debt and cash**, making it more useful than the P/E ratio in some cases.

**📌 EV/EBITDA Formula**

$$
EV/EBITDA = \frac{Enterprise\ Value}{EBITDA}
$$

Where:  
- **Enterprise Value (EV) = Market Capitalization + Debt - Cash**  
- **EBITDA = Earnings Before Interest, Taxes, Depreciation, and Amortization**  

**📌 Why Use EV/EBITDA?**
- **Accounts for Debt** → Unlike P/E, it factors in a company's debt load.  
- **Useful for Comparing Different Capital Structures** → Works well for industries where companies have varying levels of debt.  
- **Lower EV/EBITDA is Better** → A lower ratio suggests the company is undervalued relative to earnings.  

**General Interpretation:**
- **EV/EBITDA < 10** → Generally considered undervalued.  
- **EV/EBITDA between 10 - 15** → Fairly valued.  
- **EV/EBITDA > 15** → May indicate overvaluation.  

However, different industries have different standards. A **tech stock** may have a higher EV/EBITDA than a **utility company**, so it's best to compare against **industry medians**.

---

In [ ]:
df[['EVToEBITDA','PEGRatio']].describe().T

,count,mean,std,min,25%,50%,75%,max
EVToEBITDA,456.0,20.357967,36.805237,-22.57,10.690,14.900,20.255,620.59
PEGRatio,497.0,6.284181,48.435202,-158.72,1.075,1.762,2.792,844.20


In [ ]:
visualise_metrics_quantiles(df= df, metrics = ['EVToEBITDA', 'PEGRatio'], is_percent=False)

In [ ]:
df['EVToEBITDA'].quantile(0.75)

np.float64(20.255)

In [ ]:
# Compute the median EV/EBITDA for reference
ev_ebitda_median = df['EVToEBITDA'].quantile(0.75)
# df['EVToEBITDA'].median()

step2_full = df[
    (df['PEGRatio'] < 1.76) &
    (df['EVToEBITDA'] < ev_ebitda_median)
]

step2 = step1[
    (step1['PEGRatio'] < 1.76) &
    (step1['EVToEBITDA'] < ev_ebitda_median)
]


print(f"After Valuation Filter (from initial 500 companies): {step2_full.shape[0]} companies")
print(f"After Valuation Filter (after Step1): {step2.shape[0]} companies")
print(step2[['Symbol', 'PEGRatio', 'EVToEBITDA']].head())

After Valuation Filter (from initial 500 companies): 180 companies
After Valuation Filter (after Step1): 41 companies
   Symbol  PEGRatio  EVToEBITDA
5    ADBE     1.146       16.14
16   ALLE     1.740       13.47
19  GOOGL     1.126       13.65
20   GOOG     1.163       13.90
47      T     1.129        7.57


In [ ]:
step2[COLUMNS]

,Symbol,LatestQuarter,ReturnOnEquityTTM,QuarterlyEarningsGrowthYOY,QuarterlyRevenueGrowthYOY,MarketCapitalization,Beta,EVToEBITDA,PEGRatio,TrailingPE,PERatio,Sector,ProfitMargin,EVToRevenue,AnalystRatingBuy,AnalystRatingSell,AnalystTargetPrice,50DayMovingAverage,200DayMovingAverage
5,ADBE,2025-02-28,0.4730,2.044,0.103,149331952000,1.492,16.14,1.146,23.17,23.17,TECHNOLOGY,0.3060,6.750,18.0,0.0,505.31,412.54,485.05
16,ALLE,2024-12-31,0.4240,0.234,0.054,10950252000,1.021,13.47,1.740,18.63,18.63,TRADE & SERVICES,0.1580,3.285,1.0,0.0,142.39,127.84,133.93
19,GOOGL,2024-12-31,0.3290,0.309,0.118,1918377001000,1.035,13.65,1.126,19.47,19.47,TECHNOLOGY,0.2860,5.280,41.0,0.0,207.26,169.19,173.41
20,GOOG,2024-12-31,0.3290,0.309,0.118,1918520525000,1.035,13.90,1.163,19.74,19.74,TECHNOLOGY,0.2860,5.380,41.0,0.0,207.19,171.16,175.04
47,T,2024-12-31,0.1020,0.884,0.009,198020350000,0.589,7.57,1.129,18.46,18.46,TECHNOLOGY,0.0895,2.726,12.0,1.0,27.89,26.53,22.66
68,BKNG,2024-12-31,2.2570,4.091,0.144,151358145000,1.371,16.31,1.274,26.71,26.71,ENERGY & TRANSPORTATION,0.2480,6.410,21.0,0.0,5515.45,4717.64,4463.10
72,BR,2024-12-31,0.3530,1.034,0.131,27588399000,1.069,19.67,1.671,36.9,36.90,TRADE & SERVICES,0.1140,4.677,0.0,0.0,243.93,236.12,222.98
98,CHTR,2024-12-31,0.3400,0.426,0.016,48736362000,1.128,6.71,0.362,9.81,9.81,TECHNOLOGY,0.0923,2.608,9.0,2.0,400.53,358.14,351.18
115,CMCSA,2024-12-31,0.1870,0.543,0.021,130067087000,0.943,5.90,1.399,8.31,8.31,TECHNOLOGY,0.1310,1.793,11.0,0.0,42.62,35.53,38.83
138,DECK,2024-12-31,0.3980,0.190,0.171,15811815000,1.068,10.75,1.192,16.91,16.91,MANUFACTURING,0.1910,2.808,10.0,0.0,189.99,130.56,161.89


## Step 3: Check for Technical Strength
* 50-Day Moving Average > 200-Day Moving Average (bullish momentum)
* Beta between 0.8 and 1.5 (moderate risk)

---

The **Beta (β) coefficient** measures a stock’s **volatility** relative to the overall market. It helps investors understand how much risk (or price movement) they might expect compared to the broader stock index (e.g., S&P 500).  

**📌 Beta Formula**
$$
\beta = \frac{\text{Covariance of Stock Returns with Market Returns}}{\text{Variance of Market Returns}}
$$

Most platforms calculate Beta using historical data, so we typically **use Beta directly from financial data sources** rather than computing it manually.

---

**📌 How to Interpret Beta?**

| **Beta Value**  | **Meaning** | **Risk Level** | **Example Stocks** |
|----------------|------------|----------------|--------------------|
| **β < 0**     | Moves opposite to the market (inverse correlation) | Very low | Gold, bonds |
| **β = 0**     | No correlation to market movements | Low | Fixed-income assets |
| **0 < β < 1** | Moves less than the market (lower volatility) | Moderate | Utilities, consumer staples |
| **β = 1**     | Moves in line with the market | Normal | S&P 500 index funds |
| **β > 1**     | More volatile than the market | High | Tech stocks, growth stocks |
| **β > 2**     | Highly volatile, significant price swings | Very high | Small-cap growth stocks, cryptocurrencies |


**📌 Why Use Beta for Stock Selection?**
- **For Stability:** Low Beta (0.8 - 1.0) → **Defensive** stocks, less affected by market swings.  
- **For Growth Potential:** Moderate Beta (1.0 - 1.5) → **Balanced** risk-reward tradeoff.  
- **For High Risk/High Reward:** High Beta (>1.5) → **Aggressive** stocks, high volatility.  

For a **5-month investment horizon**, we typically prefer **β between 0.8 and 1.5**:
- **Avoids extreme volatility**
- **Still offers growth potential**
- **Balances risk vs. reward**

---

In [ ]:
df[['50DayMovingAverage','200DayMovingAverage','Beta']].describe().T

,count,mean,std,min,25%,50%,75%,max
50DayMovingAverage,503.0,215.249125,453.162841,9.470,67.290,122.820,233.355,7288.370
200DayMovingAverage,503.0,218.057038,480.662163,9.130,68.145,124.640,228.100,8420.010
Beta,498.0,1.015857,0.407984,0.057,0.734,1.029,1.251,2.745


In [ ]:
visualise_metrics_quantiles(df= df, metrics = ['50DayMovingAverage','200DayMovingAverage','Beta'], is_percent=False)

In [ ]:
step3_full = df[
    (df['50DayMovingAverage'] > df['200DayMovingAverage']) &
    (df['Beta'].between(0.8, 1.5))
]

step3 = step2[
    (step2['50DayMovingAverage'] > step2['200DayMovingAverage']) &
    (step2['Beta'].between(0.8, 1.5))
]

print(f"After Technical Filter (from initial 500 companies): {step3_full.shape[0]} companies")
print(f"After Technical Filter (after Step2): {step3.shape[0]} companies")
print(step3[['Symbol', '50DayMovingAverage', '200DayMovingAverage', 'Beta']].head())

After Technical Filter (from initial 500 companies): 122 companies
After Technical Filter (after Step2): 8 companies
    Symbol  50DayMovingAverage  200DayMovingAverage   Beta
68    BKNG             4717.64              4463.10  1.371
72      BR              236.12               222.98  1.069
98    CHTR              358.14               351.18  1.128
187   FFIV              279.61               236.94  1.141
197     FI              221.34               196.06  0.916


## Step 4: Analyst Sentiment Check
* More Buy Ratings than Sell Ratings
* Analyst Target Price at least 5% above current 50-day moving average

In [ ]:
df[['AnalystRatingBuy','AnalystRatingSell','AnalystRatingStrongSell','AnalystTargetPrice']].describe().T

,count,mean,std,min,25%,50%,75%,max
AnalystRatingBuy,500.0,9.004000,6.699384,0.00,5.000,8.00,12.00,48.00
AnalystRatingSell,500.0,0.452000,0.867872,0.00,0.000,0.00,1.00,9.00
AnalystRatingStrongSell,500.0,0.582000,0.829851,0.00,0.000,0.00,1.00,4.00
AnalystTargetPrice,503.0,241.383917,530.825176,10.01,76.365,137.75,253.45,8923.33


In [ ]:
#
visualise_metrics_quantiles(df= df, metrics = ['AnalystRatingBuy','AnalystRatingSell','AnalystRatingStrongSell','AnalystTargetPrice'], is_percent=False)

In [ ]:
step4_full = df[
    (df['AnalystRatingBuy'] > df['AnalystRatingSell'] + df['AnalystRatingStrongSell']) &
    (df['AnalystTargetPrice'] > df['50DayMovingAverage'])
]

step4 = step3[
    (step3['AnalystRatingBuy'] > step3['AnalystRatingSell'] + step3['AnalystRatingStrongSell']) &
    (step3['AnalystTargetPrice'] > step3['50DayMovingAverage'])
]

print(f"After Analyst Sentiment Filter (filter from the initial 500 companies): {step4_full.shape[0]} companies")
print(f"After Analyst Sentiment Filter (after Step3): {step4.shape[0]} companies")

step4[COLUMNS].head(10).T

After Analyst Sentiment Filter (filter from the initial 500 companies): 448 companies
After Analyst Sentiment Filter (after Step3): 6 companies


,68,98,197,295,311,390
Symbol,BKNG,CHTR,FI,LULU,META,RL
LatestQuarter,2024-12-31,2024-12-31,2024-12-31,2025-01-31,2024-12-31,2024-12-31
ReturnOnEquityTTM,2.257,0.34,0.109,0.424,0.371,0.276
QuarterlyEarningsGrowthYOY,4.091,0.426,0.137,0.161,0.514,0.112
QuarterlyRevenueGrowthYOY,0.144,0.016,0.068,0.127,0.206,0.108
MarketCapitalization,151358145000,48736362000,118881313000,30839642000,1321354396000,12554191000
Beta,1.371,1.128,0.916,1.313,1.279,1.395
EVToEBITDA,16.31,6.71,16.13,10.31,15.17,11.23
PEGRatio,1.274,0.362,0.977,1.054,1.065,1.559
TrailingPE,26.71,9.81,39.56,17.45,21.86,18.51


In [ ]:
# prompt: add styling to show $B for market cap: step4[COLUMNS].head().T - i need billions, and all others pretty - i mean without a lot of zeros

step4[COLUMNS].style.format({
    'MarketCapitalization': lambda x: f"${x / 1e9:,.1f}B" if pd.notna(x) else "",
    'ReturnOnEquityTTM': '{:.2f}',
    'QuarterlyEarningsGrowthYOY': '{:.2f}',
    'QuarterlyRevenueGrowthYOY': '{:.2f}',
    'Beta': '{:.2f}',
    'EVToEBITDA': '{:.2f}',
    'PEGRatio': '{:.2f}',
    'AnalystRatingBuy': '{:.0f}',
    'AnalystRatingSell': '{:.0f}',
    'AnalystTargetPrice': '{:.2f}',
    '50DayMovingAverage': '{:.2f}',
    '200DayMovingAverage': '{:.2f}'
})


,Symbol,LatestQuarter,ReturnOnEquityTTM,QuarterlyEarningsGrowthYOY,QuarterlyRevenueGrowthYOY,MarketCapitalization,Beta,EVToEBITDA,PEGRatio,TrailingPE,PERatio,Sector,ProfitMargin,EVToRevenue,AnalystRatingBuy,AnalystRatingSell,AnalystTargetPrice,50DayMovingAverage,200DayMovingAverage
68,BKNG,2024-12-31,2.26,4.09,0.14,$151.4B,1.37,16.31,1.27,26.71,26.710000,ENERGY & TRANSPORTATION,0.248000,6.410000,21,0,5515.45,4717.64,4463.10
98,CHTR,2024-12-31,0.34,0.43,0.02,$48.7B,1.13,6.71,0.36,9.81,9.810000,TECHNOLOGY,0.092300,2.608000,9,2,400.53,358.14,351.18
197,FI,2024-12-31,0.11,0.14,0.07,$118.9B,0.92,16.13,0.98,39.56,39.560000,TECHNOLOGY,0.153000,6.970000,22,0,249.70,221.34,196.06
295,LULU,2025-01-31,0.42,0.16,0.13,$30.8B,1.31,10.31,1.05,17.45,17.450000,MANUFACTURING,0.171000,2.874000,16,2,343.67,336.35,315.71
311,META,2024-12-31,0.37,0.51,0.21,"$1,321.4B",1.28,15.17,1.06,21.86,21.860000,TECHNOLOGY,0.379000,8.010000,44,0,733.99,626.83,578.37
390,RL,2024-12-31,0.28,0.11,0.11,$12.6B,1.40,11.23,1.56,18.51,18.510000,MANUFACTURING,0.101000,1.884000,8,0,281.25,240.17,209.86


In [ ]:
step4[['Symbol','OfficialSite','Description']]

,Symbol,OfficialSite,Description
68,BKNG,https://www.bookingholdings.com,Booking Holdings Inc. is an American travel te...
98,CHTR,https://corporate.charter.com,"Charter Communications, Inc., is an American t..."
197,FI,https://www.fiserv.com,"Frank's International NV, provides various tub..."
295,LULU,https://shop.lululemon.com,lululemon athletica inc. The company is headqu...
311,META,https://investor.atmeta.com,"Meta Platforms, Inc. develops products that en..."
390,RL,https://www.ralphlauren.com,Ralph Lauren Corporation is an American fashio...


In [ ]:
for i,elem in enumerate(step4.Description.to_list()):
  print(i,elem)

0 Booking Holdings Inc. is an American travel technology company organized in Delaware and based in Norwalk, Connecticut, that owns and operates several travel fare aggregators and travel fare metasearch engines including namesake and flagship Booking.com, Priceline.com, Agoda.com, Kayak.com, Cheapflights, Rentalcars.com, Momondo, and OpenTable.
1 Charter Communications, Inc., is an American telecommunications and mass media company with services branded as Charter Spectrum.
2 Frank's International NV, provides various tubular engineering services for oil and gas exploration and production, and oilfield service companies in the United States, Europe, the Middle East, Africa, Latin America, Asia Pacific and internationally. The company is headquartered in Den Helder, the Netherlands.
3 lululemon athletica inc. The company is headquartered in Vancouver, Canada.
4 Meta Platforms, Inc. develops products that enable people to connect and share with friends and family through mobile devices,

# Final Step : Deep Research from OpenAI



*   API Info: https://platform.openai.com/docs/guides/reasoning?api-mode=responses
*   List item

